In [23]:
from typing import Any, Dict, Iterator, List, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
import json
from dify_client import ChatClient
from pydantic import Field, HttpUrl

class ChatFlowLLM(LLM):
    api_key: str = Field(..., description="API key for Dify")
    user_id: str = Field(..., description="User ID for Dify")
    base_url: HttpUrl = Field(..., description="Base URL for Dify API")
    chat_client: ChatClient = Field(None, exclude=True)

    def __init__(self, **data):
        super().__init__(**data)
        self.chat_client = ChatClient(api_key=self.api_key)
        self.chat_client.base_url = str(self.base_url)

    def _call(self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None, **kwargs: Any) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        chat_response = self.chat_client.create_chat_message(inputs={}, query=prompt, user=self.user_id, response_mode="blocking")
        chat_response.raise_for_status()
        response_data = chat_response.json()
        return response_data.get('answer', '')

    def _stream(self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None, **kwargs: Any) -> Iterator[GenerationChunk]:
        chat_response = self.chat_client.create_chat_message(inputs={}, query=prompt, user=self.user_id, response_mode="streaming")
        chat_response.raise_for_status()

        for line in chat_response.iter_lines(decode_unicode=True):
            line = line.split('data:', 1)[-1]
            if line.strip():
                line = json.loads(line.strip())
                chunk = GenerationChunk(text=line.get('answer', ''))
                if run_manager:
                    run_manager.on_llm_new_token(chunk.text, chunk=chunk)
                yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model_name": "ChatFlowLLM"}

    @property
    def _llm_type(self) -> str:
        return "dify_custom"

In [25]:

# 使用例
api_key = "app-xxxxxxxxxxxxx"
user_id = "user_id"
base_url = "https://example.com"

dify_llm = DifyCustomLLM(api_key=api_key, user_id=user_id, base_url=base_url)

結果: こんにちは！今日はどんなお手伝いができますか？
ストリーム結果:
こんにちは！どうかしましたか？今日はどんなことをお話ししましょうか？

In [26]:
dify_llm.invoke("This is a foobar thing")

'It sounds like you might be referencing something specific when you mention "foobar." Could you please provide more context or specify what you mean by "foobar"? The term "foobar" is often used as a placeholder name in computer programming and other technical documentation, similar to "foo" and "bar." It might represent a variable, function, or any sort of example element. If you can give more details about the context—whether it\'s a tech problem, a coding exercise, or something else—I can offer a more targeted response.'